# <span style="color:#91299A">Основа аспирантской диссертации - проблема наблюдаемости системы</span> 

![](../storage/banners/7_observability.png)

##### <span style="color:#0ab49a">Единая система уравнений:</span> <span style="color:#A254FC">из PyCharm</span> 

In [1]:
from __init__ import *

Инициализация проекта kiam-femto | Контекст: kiamfemto


### <span style="color:#0ab49a">Свои</span> <span style="color:#A254FC">догадки</span> 

#### <span style="color:#2c3e50">Попытка вывода симметрии у решения</span> 
**Частные случаи**

In [11]:
from sympy import *
import sympy
from cosmetic import *

def add_extraname(s, extraname):
    return " ".join([i+extraname for i in s.split()])

def get_x_y_z(case: dict, extraname: str = "", planar: bool = False, test_xz: bool = False):
    x0, y0, z0, vx0, vy0, vz0 = var(add_extraname(s="x_0 y_0 z_0 v^x_0 v^y_0 v^z_0", extraname=extraname))
    X0, Y0, Z0, VX0, VY0, VZ0 = var(add_extraname(s="X_0 Y_0 Z_0 V^X_0 V^Y_0 V^Z_0", extraname=extraname))
    c, rho, m, M, s, S, v_orb = var(add_extraname(s="C rho m M s S v_{orb}", extraname=extraname))
    t, w0 = var("t w_0")

    r, R = 3*[None], 3*[None]
    
    if case['C1 = 0']:
        vx0 = - 2 * z0 * w0
        VX0 = - 2 * Z0 * w0
    if not case['CubeSat motion']:
        X0, Y0, Z0, VX0, VY0, VZ0 = [0]*6
    if planar:
        y0, Y0, vy0, VY0 = [0]*4

    for _x0, _y0, _z0, _vx0, _vy0, _vz0, _r, _m, _s in zip((x0, X0), (y0, Y0), (z0, Z0), (vx0, VX0), (vy0, VY0), (vz0, VZ0), 
                                                           (r, R), (m, M), (s, S)):
        C_1 = 2 * _z0 + _vx0 / w0
        C_2 = _vz0 / w0
        C_3 = -3 * _z0 - 2 * _vx0 / w0
        C_4 = _x0 - 2 * _vz0 / w0
        C_5 = _vy0 / w0
        C_6 = _y0
        
        tmp = (-t**2 / 2 * c * rho / _m * _s * v_orb**2) if case['Aero'] else 0
        
        _r[0] = C_4 - 3*C_1*w0*t + 2*C_2*cos(w0*t) -2*C_3*sin(w0*t) + tmp
        _r[1] = C_5*sin(w0*t) + C_6*cos(w0*t)
        _r[2] = 2*C_1 + C_2*sin(w0*t) + C_3*cos(w0*t)
    
    return {'w0': w0, 't': t,
            'x0': x0, 'y0': y0, 'z0': z0, 'vx0': vx0, 'vy0': vy0, 'vz0': vz0,
            'x': r[0], 'y': r[1], 'z': r[2], 
            'X': R[0], 'Y': R[1], 'Z': R[2]}

def get_measurements(case: dict, params: dict):
    if case['antenna type'] == 'isotropic':
        measurements = [(params['X'] - params['x'])**2 + 
                        (params['Y'] - params['y'])**2 + 
                        (params['Z'] - params['z'])**2]

    return measurements

def get_discrepancy_invert_param(measurements, params: dict, variables: list) -> list:
    global counter
    subses = []
    for v in variables:
        subses.append((params[v], -params[v]))

    anw = []
    for i, m in zip(range(len(measurements)), measurements):
        tmp = (m - m.subs(subses)).simplify()
        anw.append(tmp)
        if isinstance(tmp, sympy.core.numbers.Zero):
            print(f"По параметрам\033[1m", *variables, f"\033[0mу измерения №{i+1} есть зеркальная симметрия")
            counter *= 2
    return anw

def print_symmetry_problem_result():
    global counter
    if counter == 1:
        my_print(f"Решение единственное: {counter}!", bold=True, color="g")
    else:
        my_print(f"Симметричных решений: {counter}", bold=True, color="b")


def calculate_symmetry(motion_case, measurement_case, v_list):
    global counter
    counter = 1
    params = get_x_y_z(case=motion_case)
    measurements = get_measurements(case=measurement_case, params=params)
    for variables in v_list:
        _ = get_discrepancy_invert_param(measurements=measurements, variables=variables, params=params)
    print_symmetry_problem_result()

##### 0. Поиск решения

>> Уравнения:
$$\ddot{x} = -2 \omega_0 \upsilon_z,$$
$$\ddot{y} = -\omega_0^2 y,$$
$$\ddot{z} = 2 \omega_0 \upsilon_x + 3 \omega_0^2 z.$$

> Движение по $y$ отделимо <br>
> Возможна симметрия по одновременному изменению положений и скоростей по $x, z$ <br>

In [13]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

params = get_x_y_z(case=motion_case)
measurements = get_measurements(case=measurement_case, params=params)

# params_wrong = get_x_y_z(case=motion_case, planar=False, extraname="^w")
# measurements_wrong = get_measurements(case=measurement_case, params=params_wrong)
measurements[0]

(-v^y_0*sin(t*w_0)/w_0 - y_0*cos(t*w_0))**2 + (-2*v^x_0/w_0 - v^z_0*sin(t*w_0)/w_0 - 4*z_0 - (-2*v^x_0/w_0 - 3*z_0)*cos(t*w_0))**2 + (t*w_0*(3*v^x_0/w_0 + 6*z_0) - 2*v^z_0*cos(t*w_0)/w_0 + 2*v^z_0/w_0 - x_0 + (-4*v^x_0/w_0 - 6*z_0)*sin(t*w_0))**2

In [16]:
grad_vec = {}
for v in ["x0", "y0", "z0", "vx0", "vy0", "vz0"]:
    grad_vec[v] = measurements[0].diff(params[v])

##### 1. ХКУ, 1к-1ч, изотропные антенны, без определение углового движения
> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br>
> Итого: 4 симметричных решения

In [117]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vx0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 4


In [118]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

Решение единственное: 1!


In [119]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
По параметрам x0 z0 vz0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 8


In [120]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': False,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


##### 2. ХКУ + аэро, 1к-1ч, изотропные антенны, без определение углового движения
> Уравнения ХКУ имеют 4 симметричных решения <br>
> Антенны изотропные, но углы не ищутся -> симметрия не уменьшается <br> 
> Итого: 4 симметричных решения

In [121]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам y0 vy0 у измерения №1 есть зеркальная симметрия
Симметричных решений: 2


In [122]:
# Инициализация
motion_case = {'C1 = 0': False, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0'],
          ['x0', 'z0', 'vx0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)


KeyboardInterrupt



In [105]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': False}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
По параметрам y0 vy0 у измерения №1 есть симметрия
Симметричных решений: 4


In [106]:
# Инициализация
motion_case = {'C1 = 0': True, 
               'Aero': True,
               'CubeSat motion': True}

measurement_case = {'antenna type': 'isotropic'}

v_list = [['x0'], ['vx0'], ['y0'], ['vy0'], ['z0'], ['vz0'],
          ['x0', 'vx0'], ['y0', 'vy0'], ['z0', 'vz0'],
          ['x0', 'z0'],
          ['x0', 'z0', 'vz0']]

calculate_symmetry(motion_case=motion_case, measurement_case=measurement_case, v_list=v_list)

По параметрам vx0 у измерения №1 есть симметрия
Симметричных решений: 2


#### <span style="color:#2c3e50">Минимальное кол-во материнских аппаратов для наблюдаемости</span> 

In [9]:
def get_min_chief_amount(fn: int, motion: str, antenna_config: dict) -> int:
    """
    :param fn: кол-во дочерних КА (НЕ УЧИТЫВАЕТСЯ)
    :param motion: По каким координатам ОСК движение КА
    :param antenna_config: типы антенн материнских и дочерних КА
    :return: Кол-во материнских КА для наблюдаемости на НОО (околокруговые)
    """
    for cn in range(1, 10):
        n_symmetry_solution = [1, 1, 1]
        ############################
        # Увеличение симметричных решений
        if antenna_config['c-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        if antenna_config['d-type'] == "изотропные":
            for i in range(3):
                n_symmetry_solution[i] *= 2
        
        # Уменьшение симметричных решений
        if not antenna_config['c-multy-send'] and not antenna_config['d-multy-take']:
            if cn == 4:
                for i in range(3):
                    n_symmetry_solution[i] /= 2
        ############################
        if sum(n_symmetry_solution) == 3:
            return cn
    print("Не нашлось такого количества! Измени параметры")
    return -1
    
cn = get_min_chief_amount(fn=1, motion="xyz", 
                          antenna_config={'c-type': ["изотропные"][0], 
                                          'd-type': ["изотропные"][0],
                                          'c-multy-send': False,
                                          'c-multy-take': False,
                                          'd-multy-send': False,
                                          'd-multy-take': False,})
print(f"Минимальное кол-во материнских аппаратов для наблюдаемости: \033[1m{cn}\033[0m")

Не нашлось такого количества! Измени параметры
Минимальное кол-во материнских аппаратов для наблюдаемости: -1


#### <span style="color:#2c3e50">Закрутка чипсата</span> 

### <span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">038 (Shauying R.K.) Observability of Nonlinear Systems</span> 

#### <span style="color:#2c3e50">Начало</span> 

In [2]:
o = kf.init()
o

FileNotFoundError: [Errno 2] No such file or directory: 'kiamfemto/data/config_choose.csv'

In [6]:
!pip install kiam-astro

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.2
    Uninstalling numpy-2.1.2:
      Successfully uninstalled numpy-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipympl 0.9.3 requires ipython-genutils, which is not installed.
basemap 1.4.1 requires matplotlib<3.9,>=1.5; python_version >= "3.5", but you have matplotlib 3.9.2 which is incompatible.
basemap 1.4.1 requires packaging<24.0,>=16.0; python_version >= "3.5", but you have packaging 24.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python

In [1]:
from sympy import *
import numpy as np

# Общие параметры
h_orb = 350e3
RadiusEarth = 6371000.
Radius_orbit = RadiusEarth + h_orb
mu = 398576057600000.06
ω_orb = np.sqrt(mu / (Radius_orbit ** 3))
v_orb = ω_orb * Radius_orbit

# Расчёт плотности атмосферы -> https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html
t = -131.21 + 0.00299 * h_orb
p = 2.488 * ((t + 273.1) / 216.6) ** -11.388
ρ = p / (0.2869 * (t + 273.1))

print(f"Высота орбиты: {int(h_orb // 1e3)} км\nПериод орбиты: {round((2*np.pi/ω_orb) / 3600, 2)} часов\nПлотность атмосферы: {ρ} кг/м³")

Высота орбиты: 350 км
Период орбиты: 1.52 часов
Плотность атмосферы: 2.7797537885625094e-11 кг/м³


#### <span style="color:#2c3e50">Алгоритм</span>

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">055 (Yujiro Inowe) On the Observability of Autonomous Nonlinear Systems</span> <span style="color:#0ab49a">надо просто проверить ранг во всём $R^n$!</span>    $rg\frac{\partial \boldsymbol{H}_d}{\partial \boldsymbol{x}} (\boldsymbol{x}) = n \hskip20px \forall x \in R^n,$
$$d = n \frac{n +3}{2}.$$

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">051 (Andrew J. Whalen) Observability and Controllability of Nonlinear Networks The Role of Symmetry</span> <span style="color:#0ab49a">надо проверить</span> $\delta(x) = \frac{|\sigma_{min}[O^T O]|}{|\sigma_{max}[O^T O]|}.$

In [2]:
# Функции вспомогательные
def matrix_minor(arr: np.ndarray, i: int, j: int) -> float:
    return np.linalg.det(np.delete(np.delete(arr,i,axis=0), j, axis=1))
        
def quat2matrix(L) -> Matrix:
    """Преобразует единичный кватернион в матрицу поворота
    :param q: Кватернион, list длинны 4
    :return: Матрица поворота, sympy.Matrix"""
    w, x, y, z = L
    A = Matrix(np.eye(3))
    A[0, 0] = 1 - 2 * y ** 2 - 2 * z ** 2
    A[0, 1] = 2 * x * y + 2 * z * w
    A[0, 2] = 2 * x * z - 2 * y * w
    A[1, 0] = 2 * x * y - 2 * z * w
    A[1, 1] = 1 - 2 * x ** 2 - 2 * z ** 2
    A[1, 2] = 2 * y * z + 2 * x * w
    A[2, 0] = 2 * x * z + 2 * y * w
    A[2, 1] = 2 * y * z - 2 * x * w
    A[2, 2] = 1 - 2 * x ** 2 - 2 * y ** 2
    return A

def local_dipol(r: list, r_abs, ind: str, q: list, distortion=0):
    """Возвращает усиление антенны полуволногого диполя
    :param r: направление на аппарат в орбитальной системе координат (ОСК)
    :param r_abs: модуль вектора r (для таких лентяев как йа)
    :param ind: ось x/y/z, по которой направлена антенна в собственной системе координат (ССК)
    :param q: кватернион поворота ОСК -> ССК
    :param distortion: искажение диаграммы направленности (несиммеричность)"""
    r_ = quat2matrix(q) @ Matrix(r)
    cos_a = (r_[0]*int(ind=='x') + r_[1]*int(ind=='y') + r_[2]*int(ind=='z'))/r_abs
    sin_a = (sqrt(r_[1]**2 + r_[2]**2)*int(ind=='x') + \
             sqrt(r_[0]**2 + r_[2]**2)*int(ind=='y') + \
             sqrt(r_[0]**2 + r_[1]**2)*int(ind=='z'))/r_abs
    aside = ((r[1]+r[2])*int(ind=='x') + \
             (r[0]+r[2])*int(ind=='y') + \
             r[2]*int(ind=='z'))/r_abs
    return cos(cos_a * pi / 2) / sin_a + distortion * cos_a**2 + distortion * aside

def get_vars(name: str, n: int):
    """Генерит символьные переменные с одинаковым названием и индексами 0...(n-1)
    :param name: Название переменных (без учёта индекса)
    :param n: Количество переменных"""
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

def get_func(name: str, n: int):
    """Генерит символьные функции с одинаковым названием и индексами 0...(n-1)
    :param name: Название функций (без учёта индекса)
    :param n: Количество функций"""
    return [Function(f"{name}_{i}", real=True)(t) for i in range(n)]

def get_params(n: int):
    """Экспресс-инициализация переменных и функций для исследования критериев наблюдаемости
    :param n: Количество чипсатов, кинематические параметры которых нужно найти/оценить"""
    t, ω = var("t ω", real=True)
    r_cube = get_func(f"r^c", 3)
    v_cube = get_func(f"v^c", 3)
    x = get_func(f"x", n)
    y = get_func(f"y", n)
    z = get_func(f"z", n)
    vx = get_func(f"v^x", n)
    vy = get_func(f"v^y", n)
    vz = get_func(f"v^z", n)
    wx = get_func(f"w^x", n)
    wy = get_func(f"w^y", n)
    wz = get_func(f"w^z", n)
    q0 = get_func(f"q^0", n)
    qx = get_func(f"q^x", n)
    qy = get_func(f"q^y", n)
    qz = get_func(f"q^z", n)
    return t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz

def save_reports(report_list: list, filename: str) -> None:
    """Сохраняет в компактном виде несколько выводов в один
    :param report_list: Строки вывода, запрашиваемые к объединению
    :param filename: Название текстовика, в котором будет храниться информация"""
    common_report = report_list[0]
    for i in range(len(report_list) - 1):
        common_report += ("\n" + "-" * 100)*2 + "\n" + report_list[i + 1]
    f = open("storage/observability_reoprt_" + filename + ".txt", "w")
    f.write(common_report)
    f.close()

def read_reports(filename: str) -> str:
    """Считывает выводы, сохранённые в один файл
    :param filename: Название текстовика, в котором хранится информация"""
    f = open("storage/observability_reoprt_" + filename + ".txt", "r")
    common_report = f.read()
    f.close()
    return common_report

def print_spectrm_rank(J_numb):
    for tol in [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15, 1e-20]:
        print(f"Ранг матрицы: {np.linalg.matrix_rank(J_numb, tol=tol)} (tol={tol})")

def SubRandParams(J, n: int, n_x: int, n_y: int, testprint: bool = False):
    """Берёт матрицу J размером n_x на n_y, подставляет случайные значения"""
    # Генерация случайных параметров движения
    s_r = lambda: np.random.uniform(-100, 100)
    s_v = lambda: np.random.uniform(-10, 10)
    s_w = lambda: np.random.uniform(-1e-4, 1e-4)
    q = np.array([s_v() for _ in range(4)])
    q /= np.linalg.norm(q)
    rand_params = [(ω, ω_orb), (pi, np.pi), 
                   (r_cube[0], s_r()), (r_cube[1], s_r()), (r_cube[2], s_r()), 
                   (v_cube[0], s_v()), (v_cube[1], s_v()), (v_cube[2], s_v())]
    for i in range(n):
        tmp_r = [(x[i], s_r()), (y[i], s_r()), (z[i], s_r())]
        tmp_v = [(vx[i], s_v()), (vy[i], s_v()), (vz[i], s_v())]
        rand_params += tmp_r + tmp_v + [(wx[i], s_w()), (wy[i], s_w()), (wz[i], s_w()), 
                                        (q0[i], q[0]), (qx[i], q[1]), (qy[i], q[2]), (qz[i], q[3])]

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J_numb = np.array([[0. for _ in range(n_x)] for _ in range(n_y)])
    for i in range(n_y):
        if testprint:
            print(f"_J_numb: расчёт строки_: {i+1} / {n_y}")
        for j in range(n_x):
            J_numb[i][j] = float(J[i, j].subs(rand_params))
    return J_numb

In [26]:
def ShauyingObservabilitySufficientCondition(n: int, X: list, Y: list, my_diff, testprint: bool = False, hand_written_deriv: int = None):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени
    :param testprint: Флаг вывода экстра-информации"""
    report = f"\033[1mКоличество чипсатов: {n}\033[0m\n"
    print(report)

    def LocalPrint(report: str, lcl_txt: str):
        report += lcl_txt + "\n"
        print(lcl_txt)
        return report

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    d = len(X) * (len(X) + 3) / 2  / len(Y)
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    report = LocalPrint(report, txt + f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)\nКритерий (055): происзводные порядка {d}")
    
    # Матрица наблюдаемости системы
    H = Matrix([[Y[0] for ll in range(l)] for kk in range(k)])
    H_one_line = []
    for kk in range(k):
        for ll in range(l):
            tmp = Y[ll] if kk == 0 else my_diff(H[kk - 1, ll])
            if testprint:
                print(f"_расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H[kk, ll] = tmp
            H_one_line += [tmp]
    H = Matrix(H_one_line)
    report = LocalPrint(report, f"Размерность матрицы H: {shape(H)}")

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[H[i].diff(X[j]) for j in range(len(X))] for i in range(k * l)])
    report = LocalPrint(report, f"Размерность матрицы J: {shape(J)}")

    # Подстановка конкретных значений
    J_numb = SubRandParams(J=J, n=n, n_x=len(X), n_y=k*l, testprint=testprint)
    _, v, _ = np.linalg.svd(J_numb.T @ J_numb)
    report = LocalPrint(report, f"σₘₙ/σₘₐₓ = {np.min(v)}/{np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁵: {np.sum(v>1e-5)}/{len(v)} (статья 051)")
    # H_H_numb = SubRandParams(J=np.outer(H, H), n=n, n_x=k*l, n_y=k*l, testprint=False)
    # _, v, _ = np.linalg.svd(H_H_numb)
    # report = LocalPrint(report, f"σₘₙ/σₘₐₓ = {np.min(v)} / {np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁵: {np.sum(v>1e-5)}/{len(v)} (статья 060)")
    
    # Достаточное условие
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]]} (статья 055)\n"
    txt += f"Детерминант матрицы: {np.linalg.det(J_numb)}\n" if J_numb.shape[0] == J_numb.shape[1] else ""
    if hand_written_deriv is None:
        report = LocalPrint(report, txt + f"Следующие параметры не должны быть нулевыми:\n")
        d, Δ, flag, i_min = ([], [], True, -1)
        for i in range(len(X)):
            tmp = matrix_minor(J_numb, i, i)
            d += [tmp if i == 0 else tmp / Δ[-1]]
            Δ += [tmp]
            report = LocalPrint(report, f"Δ_{i} = {Δ[-1]}")
        
            # Чек наблюдаемости
            if flag:
                if abs(d[-1]) < 1e-6:
                    i_min = i
                    flag = False
            if not flag:
                break
    
        # Вывод
        if flag:
            txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
        else:
            in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
            txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
        report = LocalPrint(report, txt)
        return H, J, J_numb, Δ, report
    report = LocalPrint(report, txt)
    return H, J, J_numb, report

### <span style="color:#00b90e">Применение алгоритма на примерах</span>

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#bb4bca">при кубсате строго на круговой орбите, без аэродинамики и углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [27]:
# Уравнения движения
def MyDiff_AeroOff_AttitudeOff(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: НЕ УЧИТЫВАЕТСЯ
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [39]:
# Количество чипсатов
# {1->6, 5->2, 11->1}
# {1->4, 3->2, 7->1} без у
n = 11

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
    # X += [x[i], z[i], vx[i], vz[i]]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]

H_1, J_1, J_numb_1, d_1, report_1_1 = ShauyingObservabilitySufficientCondition(testprint=False, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOff_AttitudeOff)

Количество чипсатов: 11

Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
Критерий (055): происзводные порядка 34.5
Размерность матрицы H: (66, 1)
Размерность матрицы J: (66, 66)
σₘₙ/σₘₐₓ = 7.684434599837031e-16/11.989042513244367 = 6.409548211500618e-17 | σ>10⁻⁵: 30/66 (статья 051)

Ранг матрицы: [30, 30, 30, 30, 30, 30] (статья 055)
Детерминант матрицы: 0.0
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0


In [ ]:
# save_reports([report_1_1, report_1_5, report_1_11], "AeroOff_AttitudeOff_AntennaOff")
print(read_reports("AeroOff_AttitudeOff_AntennaOff"))

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой,</span> <span style="color:#bb4bca">без углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [28]:
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм

# Уравнения движения
def MyDiff_AeroOn_AttitudeOff(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2)]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [40]:
# Количество чипсатов
# {1->6, 5->2, 11->1}
# {1->4, 3->2, 7->1} без у
n = 1

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
    # X += [x[i], z[i], vx[i], vz[i]] 

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]
            
H_11, J_11, J_numb_11, d_11, report_11_11 = ShauyingObservabilitySufficientCondition(testprint=False, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff)
# H_11, J_11, J_numb_11, report_11_11 = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=8, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff)
# report_11_7 = f"\033[1mВнимание! Нет учёта компоненты y!\033[0m\n" + report_11_

Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 1
∃ производные порядка k = 6.0 (Должна быть целой!)
Критерий (055): происзводные порядка 27.0
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
σₘₙ/σₘₐₓ = 5.820990780442059e-18/1.0195921275403392 = 5.709136647106719e-18 | σ>10⁻⁵: 3/6 (статья 051)

Ранг матрицы: [3, 4, 4, 6, 6, 6] (статья 055)
Детерминант матрицы: 1.1232175608345237e-23
Следующие параметры не должны быть нулевыми:

Δ_0 = -8.156175146517246e-18

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = -8.156175146517246e-18


In [ ]:
# save_reports([report_11_1, report_11_5, report_11_11, report_11_1_, report_11_3_, report_11_7_], "AeroOn_AttitudeOff_AntennaOff")
print(read_reports("AeroOn_AttitudeOff_AntennaOff"))

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате</span> <span style="color:#ed9a00">не на круговой орбите,</span> <span style="color:#00b0b9">с аэродинамикой,</span> <span style="color:#bb4bca">без углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [19]:
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм

# Уравнения движения
def MyDiff_AeroOn_AttitudeOff_WithCubeSat(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = [(Derivative(r_cube[0], t), v_cube[0]), 
              (Derivative(r_cube[1], t), v_cube[1]), 
              (Derivative(r_cube[2], t), v_cube[2]),
              (Derivative(v_cube[0], t), -2*ω*v_cube[2] - (0.3)**2*C*ρ/10 * (v_orb + v_cube[1])**2),
              (Derivative(v_cube[1], t), -ω**2*r_cube[1]),
              (Derivative(v_cube[2], t), 2*ω*v_cube[0] + 3*ω**2*r_cube[2])]
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2)]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [21]:
# Количество чипсатов
# {1->6, 5->2, 11->1}
# {1->4, 3->2, 7->1} без у
n = 1

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], z[i], y[i], vx[i], vy[i], vz[i]]
    # X += [x[i], z[i], vx[i], vz[i]]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt((x[i]-r_cube[0])**2 + (y[i]-r_cube[1])**2 + (z[i]-r_cube[2])**2)]
            # Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]
            
H_12, J_12, J_numb_12, d_12, report_12_1 = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff_WithCubeSat)
# H_12, J_12, J_numb_12, report_12_ = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=1, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff_WithCubeSat)
# report_12_1 = f"\033[1mВнимание! Нет учёта компоненты y!\033[0m\n" + report_12_

Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 1
∃ производные порядка k = 6.0 (Должна быть целой!)
_расчёт матрицы H_: k=1/6, l=1/1
_расчёт матрицы H_: k=2/6, l=1/1
_расчёт матрицы H_: k=3/6, l=1/1



KeyboardInterrupt



In [49]:
_, v, _ = np.linalg.svd(J_numb_12.T @ J_numb_12)
print(f"σₘₙ/σₘₐₓ = {np.min(v)} / {np.max(v)} = {np.min(v) / np.max(v)} (статья 051)")

σₘₙ/σₘₐₓ = 1.9901393773271942e-13 / 0.9297096724645358 = 2.1406030681079193e-13 (статья 051)


In [48]:
# save_reports([report_12_1, report_12_11], "AeroOn_AttitudeOff_AntennaOff_WithCubeSat")
print(read_reports("AeroOn_AttitudeOff_AntennaOff_WithCubeSat"))

Внимание! Нет учёта компоненты y!
Количество чипсатов: 1
Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 1
∃ производные порядка k = 4.0 (Должна быть целой!)
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)
σₘₙ/σₘₐₓ = 4.459350831250673e-07 / 0.9642145365345494 = 4.62485335190846e-07 (статья 051)

Ранг матрицы: [3, 3, 4, 4, 4, 4] (статья 055)
Следующие параметры не должны быть нулевыми:

Δ_0 = -1.9307937971540438e-05
Δ_1 = -0.001404428776027749
Δ_2 = 1.992862790753914e-05
Δ_3 = 0.006330687609678351

Выполнено достаточное условие! Система наблюдаема

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Количество чипсатов: 11
Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
Размерность матрицы H: (66, 1)
Размерность матрицы J: (66, 66)


----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой и угловым движением,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [ ]:
# Уравнения движения
def MyDiff_AeroOn_AttitudeOn(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: учитывается
    Примечание: крутящий момент равен 0 (даже гравитационного нет)
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        M = np.zeros(3)
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2 * (qx[i]*qy[i] - q0[i]*qz[i]))]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
        subses += [(Derivative(q0[i], t), (-wx[i]*qx[i] - wy[i]*qy[i] - wz[i]*qz[i])/2)]
        subses += [(Derivative(qx[i], t), (wx[i]*q0[i] + wy[i]*qz[i] - wz[i]*qy[i])/2)]
        subses += [(Derivative(qy[i], t), (wy[i]*q0[i] + wz[i]*qx[i] - wx[i]*qz[i])/2)]
        subses += [(Derivative(qz[i], t), (wz[i]*q0[i] + wx[i]*qy[i] - wy[i]*qx[i])/2)]
        subses += [(Derivative(wx[i], t), (Jyy*wy[i]*wz[i] - Jzz*wy[i]*wz[i] + M[0]) / Jxx)]
        subses += [(Derivative(wy[i], t), (-Jxx*wx[i]*wz[i] + Jzz*wx[i]*wz[i] + M[1]) / Jyy)]
        subses += [(Derivative(wz[i], t), (Jxx*wx[i]*wy[i] - Jyy*wx[i]*wy[i] + M[2]) / Jzz)]
    return anw.diff(t).subs(subses).simplify()

In [ ]:
# Количество чипсатов
# {1->12, 3->6, 5->4, 7->3, 11->2, 23->1}
n = 23

# Коэффициент сопротивления
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]

# H_2, J_2, J_numb_2, d_2, report_2_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)
H_2, J_2, J_numb_2, d_2, report_2_ = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=2, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)

In [ ]:
# save_reports([report_2_], "AeroOn_AttitudeOn_AntennaOff")
print(read_reports("AeroOn_AttitudeOn_AntennaOff"))

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите,</span> <span style="color:#00b0b9">с аэродинамикой и угловым движением,</span> <span style="color:#15cbfc">антенны анизотропны</span>

In [ ]:
# Количество чипсатов
# {11->1}
n = 11

# Разложение по антеннам
antennas = "xy"
print(f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m")

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, r, Y = ([], [], [])
for i in range(n):  
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
            Y += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) for c in antennas]
        else:
            tmp = sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)
            Y += [tmp / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) \
                      / sqrt(local_dipol([x[j]-x[i], y[j]-y[i], z[j]-z[i]], tmp, c, [q0[j], qx[j], qy[j], qz[j]])) for c in antennas]

H_3, J_3, J_numb_3, d_3, report_3_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)
report_3_ = f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m\n" + report_3_

In [ ]:
# save_reports([report_3_13], "AeroOn_AttitudeOn_AntennaOn")
print(read_reports("AeroOn_AttitudeOn_AntennaOn"))

----

#### <span style="color:#2c3e50">То же самое, но</span> <span style="color:#00b0b9">диаграмма направленностей не симметрична</span>

In [ ]:
# Количество чипсатов
n = 11

# Разложение по антеннам
antennas = "xy"
print(f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m")

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)
distortion = 0.4

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, r, Y = ([], [], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
            Y += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]], distortion=distortion)) for c in antennas]
        else:
            tmp = sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)
            Y += [tmp / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[i], qx[i], qy[i], qz[i]], distortion=distortion)) \
                      / sqrt(local_dipol([x[j]-x[i], y[j]-y[i], z[j]-z[i]], tmp, c, [q0[j], qx[j], qy[j], qz[j]], distortion=distortion)) for c in antennas]

H_4, J_4, J_numb_4, d_4, report_4_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)
report_4_ = f"\033[1mУ каждого чипсата {len(antennas)} кривые антенн(ы): {antennas}\033[0m\n" + report_4_

In [ ]:
J_numb_4[0][1]

In [ ]:
# save_reports([report_4_9xyz, report_4_13xy], "AeroOn_AttitudeOn_AntennaOn_CurveRadiation")
print(read_reports("AeroOn_AttitudeOn_AntennaOn_CurveRadiation"))

### <span style="color:#0ab49a">Что-то с производной</span> <span style="color:#A254FC">Ли</span>

In [ ]:
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
def GetF():
    anw = []
    for i in range(n):
        anw += [vx[i], vy[i], vz[i]]
        anw += [2*ω*vz[i], -ω**2*y[i], 2*ω*vx[i] + 3*ω**2*z[i]]
    return Matirx(anw)

# Уравнения движения
def GetGradient_AeroOff_AttitudeOff_AntennaOff(expr):
    """Функция взятия производной по времени.
    Аэродинамика: НЕ УЧИТЫВАЕТСЯ
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return expr.diff(vari).subs(subses).simplify()